In [1]:
from llama_index.vector_stores.weaviate import WeaviateVectorStore
from llama_index.core import VectorStoreIndex
from llama_index.core.response.pprint_utils import pprint_source_node
import weaviate
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

client: weaviate.WeaviateClient = weaviate.connect_to_local()
Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)
Settings.llm = Ollama(model="llama3:8b", request_timeout=60.0)

In [2]:
vector_store = WeaviateVectorStore(
    weaviate_client=client, index_name="Pymupdf", text_key="content"
)

In [3]:
retriever = VectorStoreIndex.from_vector_store(vector_store).as_retriever(
    similarity_top_k=5
)

In [4]:
from typing import List
from llama_index.core.schema import NodeWithScore

# query: str = input("Please type your query: ")
query = "what was the maek and model of the EV sophie was interested in purchasing?"
query = "i am interested in purchasing a new car. tell me a little about plug in hybrids"
query = "what are PHEVs?"
query = "What two types of motors do Plug-in hybrids (or PHEVs) have?"
nodes: List[NodeWithScore] = retriever.retrieve(query)

In [5]:
from langchain_core.prompt_values import PromptValue
from llama_index.core.base.llms.types import ChatMessage

context: str = ""
for n in nodes:
    context = context + f"""
    --------------------
    Exerpt from file with name: {n.node.metadata["properties"]["file_name"]}

    {n.text}
    --------------------
    """
prompt: list[ChatMessage] = [
    ChatMessage(role="system", content="You are an AI assistant that strictly relies on the provided context to answer user queries. Use only the information given in the context section to generate a response. If the context does not contain relevant information, inform the user accordingly. Do not use any prior knowledge."),
    ChatMessage(role="user", content="""Using the following context, generate a response to the query.
### Query:
{query}

### Context:
{context}""".format(query=query, context=context)),
]

In [6]:
response = Settings.llm.chat(prompt)

In [7]:
print(response)
print(context)

assistant: Based on the provided context, Plug-in hybrid (PHEV) vehicles have a electric motor and an internal combustion engine (ICE).

    --------------------
    Exerpt from file with name: Electric and hybrid cars.txt

    Maintaining a PHEV hybrid car
We do need to mention maintenance, though. While plug-ins have
similar benefits to EVs in cost and carbon savings, their petrol
engines (with many moving parts) do need to be taken care of. That’s where fully electric vehicles come up trumps. What are the best plug-in hybrid cars? There are a decent number of options for buying a plug-in hybrid in
New Zealand, both new and secondhand.
    --------------------
    
    --------------------
    Exerpt from file with name: Electric and hybrid cars.txt

    Here’s a breakdown of the
most popular PHEVs, and what they’ll cost you. Pros and cons of PHEV hybrid ownership
Pros
Cons 
• Can go fully electric
• Fuel efficient
• Limited electric range
• More expensive upfront
• Still need petrol

In [8]:
client.close()